### Setup

In [1]:
from google.cloud import pubsub_v1
import json
import requests
import mysql.connector
import os
from datetime import datetime

### Publiser

In [10]:
topic_path = 'projects/auditpulse/topics/deployment-request-queue'
data = {
         'run_id': "81102",
         'user_id':"1023",
         'central_index_key': '320193',
         'company_name':"AAPL",
         'ticker':'Apple Inc.',
         'year':2025
        }

jsonstring = json.dumps(data)
jsonstring = jsonstring.encode('utf-8')
publisher = pubsub_v1.PublisherClient()
future = publisher.publish(topic_path, jsonstring)

### Subscriber

In [51]:
def sub_callback(message):
    print(f'Data Received: {message}')
    global msg
    msg = message
    message.ack()
msg = None
subscriber_path = 'projects/auditpulse/subscriptions/deployment-request-queue-sub'
subscriber = pubsub_v1.SubscriberClient()
subscriber_future = subscriber.subscribe(subscriber_path, callback=sub_callback)
result = None
with subscriber:
    try:
        result = subscriber_future.result(timeout=2)
    except:
        subscriber_future.cancel()
        result = subscriber_future.result()

In [6]:
msg.data

b'{"run_id": "81102", "user_id": "1023", "central_index_key": "320193", "company_name": "AAPL", "ticker": "Apple Inc.", "year": 2025}'

In [7]:
json.loads(msg.data)

{'run_id': '81102',
 'user_id': '1023',
 'central_index_key': '320193',
 'company_name': 'AAPL',
 'ticker': 'Apple Inc.',
 'year': 2025}

In [4]:
subscriber = pubsub_v1.SubscriberClient()
subscriber_path = 'projects/auditpulse/subscriptions/deployment-request-queue-sub'

resp = subscriber.pull(
                request={
                    "subscription": subscriber_path,
                    "max_messages": 1,
                    "return_immediately": True,  # long poll
                },
            )
subscriber.acknowledge({"subscription": subscriber_path, "ack_ids": [resp.received_messages[0].ack_id]})

In [5]:
resp

received_messages {
  ack_id: "UAYWLF1GSFE3GQhoUQ5PXiM_NSAoRRcHCBQFfH1wU1R1XV98aFENGXJ9YHxsW0cBVBYBeQlRGwdoTm11H7eh9f1LQ1RrWRsJBUxaeFtfHQtgWFl1A3nI1ImDmPWQcgk9OqP_ktBtO6XDtI1EZiM9XxJLLD5-JzlFQV5AEkwsDERJUytDCypYEU4EISE-MD5FU0Q"
  message {
    data: "{"run_id": "81102", "user_id": "1023", "central_index_key": "320193", "company_name": "AAPL", "ticker": "Apple Inc.", "year": 2025}"
    message_id: "13887996446684504"
    publish_time {
      seconds: 1744244296
      nanos: 578000000
    }
  }
}

In [59]:
resp

In [6]:
print(resp.received_messages[0].ack_id,'\n',
resp.received_messages[0].message.data)

UAYWLF1GSFE3GQhoUQ5PXiM_NSAoRRcHCBQFfH1wU1R1XV98aFENGXJ9YHxsW0cBVBYBeQlRGwdoTm11H7eh9f1LQ1RrWRsJBUxaeFtfHQtgWFl1A3nI1ImDmPWQcgk9OqP_ktBtO6XDtI1EZiM9XxJLLD5-JzlFQV5AEkwsDERJUytDCypYEU4EISE-MD5FU0Q 
 b'{"run_id": "81102", "user_id": "1023", "central_index_key": "320193", "company_name": "AAPL", "ticker": "Apple Inc.", "year": 2025}'


### Update Table

In [6]:
mysql_conn = mysql.connector.connect(
    host='34.46.191.121',
    port=3306,
    user='root',
    database='auditpulse',
    password=os.getenv('MYSQL_GCP_PASS')
)

In [9]:
mysql_cursor = mysql_conn.cursor()

query = """
INSERT INTO runs 
(run_id, run_at, user_id, cik, ticker, company_name, year, status)
VALUES 
(%s, %s, %s, %s, %s, %s, %s, %s)
"""

# Current timestamp for run_at
current_time = datetime.now()

# Values to insert
values = (
    "81102",
    current_time,
    "1023",
    "320193",
    "AAPL",
    "Apple Inc.",
    2025,
    "queued"
)

mysql_cursor.execute(query,values)
mysql_conn.commit()

### Request

In [3]:
data = {
         'run_id': "81102",
         'user_id':"1023",
         'central_index_key': '320193',
         'company_name':"AAPL",
         'ticker':'Apple Inc.',
         'year':2025
        }

jsonstring = json.dumps(data)
# jsonstring = jsonstring.encode('utf-8')

In [4]:
endpoint = 'http://127.0.0.1:5000/generate'
# endpoint = 'https://auditpulse-app-853525367358.us-central1.run.app/generate'
response = requests.post(endpoint, json={'message':{'data':jsonstring}})

In [ ]:
response.text

'{\n  "message": "Report generated!",\n  "status": "Success!"\n}\n'

In [ ]:
{'message': {'data': 'eyJydW5faWQiOiAiODExMDIiLCAidXNlcl9pZCI6ICIxMDIzIiwgImNlbnRyYWxfaW5kZXhfa2V5IjogIjMyMDE5MyIsICJjb21wYW55X25hbWUiOiAiQUFQTCIsICJ0aWNrZXIiOiAiQXBwbGUgSW5jLiIsICJ5ZWFyIjogMjAyNX0=', 'messageId': '13883910100720655', 'message_id': '13883910100720655', 'publishTime': '2025-04-08T19:05:13.825Z', 'publish_time': '2025-04-08T19:05:13.825Z'}, 'subscription': 'projects/auditpulse/subscriptions/deployment-request-queue-push-sub'}{'message': {'data': 'eyJydW5faWQiOiAiODExMDIiLCAidXNlcl9pZCI6ICIxMDIzIiwgImNlbnRyYWxfaW5kZXhfa2V5IjogIjMyMDE5MyIsICJjb21wYW55X25hbWUiOiAiQUFQTCIsICJ0aWNrZXIiOiAiQXBwbGUgSW5jLiIsICJ5ZWFyIjogMjAyNX0=', 'messageId': '13879306287395948', 'message_id': '13879306287395948', 'publishTime': '2025-04-08T01:59:25.329Z', 'publish_time': '2025-04-08T01:59:25.329Z'}, 'subscription': 'projects/auditpulse/subscriptions/deployment-request-queue-push-sub'}